# Introduction
Paper: https://www.sciencedirect.com/science/article/pii/S0092867423014459

- **Subject**: mice, transgenic
- **Number**: 28

### Recording details
- **Recording type**: acute electrophysiology via Neuropixels 1.0 probes
    - this means that probes are inserted at the beginning of every session and retrieved at the end. recordings will not be taken from the exact same location twice. mice can usually be recorded from for up to 5 sessions using this technique, after which the insertion windows are no longer viable
    - ALSO: intermittent optogenetic inactivation of ALM
- **Probe density**: 2-5 simultaneous probes per recording
    - 2 probes - 2 sessions
    - 3 probes - 53 sessions
    - 4 probes - 98 sessions
    - 5 probes - 20 sessions
- **Recording targets**: anterior lateral motor cortex (ALM) circuit. individual probes target the following groups: 
    1. ALM and underlying cortical regions (e.g. orbito frontal cortex)
    2. striatum
    3. higher-order thalamus
    4. midbrain (superior colliculus (SCm), midbrain reticular nucleus (MRN), and substantia nigra pars reticulata (SNr))
    5. medulla and overlying cerebellum
    6. other
- **Spike-sorting**: Kilosort2 (probably MATLAB)
- **Unit count**: 70,000 total single units, localized using hisological information and electrophysiological targets. Median of 393 simultaneously recorded units per session.
    
### Behavior details
- **Headfixed task**: Memory-guided movement task (i.e. auditory delayed response task)
    - instruction stimuli: one of two pure tones (3 kHz or 12 kHz) played three times, 150 ms pulses and 100 ms inter-tone-interval, 650 ms total
    - delay epoch: 1.2s
    - can't lick until auditory 'Go' cue, 6 kHz carrier frequency with 360 Hz modulation, 0.1 s duration, where early licking triggered replay of delay epoch
    - response epoch: 1.5 s, correct lick triggered small water reward
    - incorrect licks triggered a 1-3 s timeout
    - trial ends after mouse stops licking for 1.5 s, followed by a 250 ms inter-trial-interval
    - early lick and no-response trials excluded from analysis
- **Video tracking**: 300Hz recording from two cameras to capture animal movements
    - offline tracking of tongue, jaw, and nose using DeepLabCut

# The data

## Acquire / Download

DANDI archive: https://dandiarchive.org/dandiset/000363/0.230822.0128

**size**: 53.6 GB

download to lab group folder? 

instructions outside of notebook?

## File structure
- 28 folders - one for each mouse
    - `.nwb` files: one for each session with naming scheme `sub-{number}_ses-{YYYYMMDD}T{HHMMSS}_behavior+ecephys[+ogen]` (most files have `+ogen` on the end, signalizing optogenetics done that session)
    
NWB file:


In [7]:
import pynacollada as nac
nac.config["data_dir"] = "../data"
data = nac.load_data("mesoscale_activity")
data

/opt/homebrew/Caskroom/miniforge/base/envs/test/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.7.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/homebrew/Caskroom/miniforge/base/envs/test/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/opt/homebrew/Caskroom/miniforge/base/envs/test/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.4.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


root pynwb.file.NWBFile at 0x4751829184
Fields:
  acquisition: {
    BehavioralEvents <class 'pynwb.behavior.BehavioralEvents'>,
    BehavioralTimeSeries <class 'pynwb.behavior.BehavioralTimeSeries'>
  }
  devices: {
    18005107442 (neuropixels 1.0 - 3B) <class 'pynwb.device.Device'>,
    18005123392 (neuropixels 1.0 - 3B) <class 'pynwb.device.Device'>,
    18194809181 (neuropixels 1.0 - 3B) <class 'pynwb.device.Device'>,
    18194822541 (neuropixels 1.0 - 3B) <class 'pynwb.device.Device'>,
    18408406312 (neuropixels 1.0 - 3B) <class 'pynwb.device.Device'>,
    OBIS470 <class 'pynwb.device.Device'>
  }
  electrode_groups: {
    18005107442 1-384 <class 'pynwb.ecephys.ElectrodeGroup'>,
    18005123392 1-384 <class 'pynwb.ecephys.ElectrodeGroup'>,
    18194809181 1-384 <class 'pynwb.ecephys.ElectrodeGroup'>,
    18194822541 1-384 <class 'pynwb.ecephys.ElectrodeGroup'>,
    18408406312 1-384 <class 'pynwb.ecephys.ElectrodeGroup'>
  }
  electrodes: electrodes <class 'hdmf.common.table.DynamicTable'>
  experiment_description: high tone vs. low tone
  experimenter: ['susu']
  file_create_date: [datetime.datetime(2023, 8, 18, 14, 35, 45, 934185, tzinfo=tzoffset(None, -18000))]
  identifier: SC052_20210129_132207_s10
  institution: Janelia Research Campus
  intervals: {
    trials <class 'pynwb.epoch.TimeIntervals'>
  }
  keywords: <StrDataset for HDF5 dataset "keywords": shape (1,), type "|O">
  ogen_sites: {
    OBIS470_4 <class 'pynwb.ogen.OptogeneticStimulusSite'>,
    OBIS470_5 <class 'pynwb.ogen.OptogeneticStimulusSite'>,
    OBIS470_6 <class 'pynwb.ogen.OptogeneticStimulusSite'>
  }
  related_publications: ['']
  session_start_time: 2021-01-29 13:22:07-06:00
  subject: subject pynwb.file.Subject at 0x4753460528
Fields:
  age__reference: birth
  date_of_birth: 2020-09-06 00:00:00-05:00
  description: SC052
  sex: F
  species: Mus musculus
  subject_id: 480928

  timestamps_reference_time: 2021-01-29 13:22:07-06:00
  trials: trials <class 'pynwb.epoch.TimeIntervals'>
  units: units <class 'pynwb.misc.Units'>

# Pynapple

## Importing the data

### Load using pynapple (allows for lazy loading)

can load directly using `nap.load_file()`, which does a good job

In [10]:
data = nap.load_file(os.path.join(fpath,fname))
print(data)

/mnt/home/svenditto/python/nsl-data/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.7.0 because version 1.8.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/mnt/home/svenditto/python/nsl-data/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'core' version 2.6.0-alpha because version 2.7.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
/mnt/home/svenditto/python/nsl-data/lib/python3.10/site-packages/hdmf/spec/namespace.py:535: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.4.0 because version 0.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


sub-441666_ses-20190513T144253_behavior+ecephys+ogen
┍━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                        │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ units                       │ TsGroup     │
│ trials                      │ IntervalSet │
│ Camera0_side_TongueTracking │ TsdFrame    │
│ Camera0_side_NoseTracking   │ TsdFrame    │
│ Camera0_side_JawTracking    │ TsdFrame    │
│ trialend_stop_times         │ Tsd         │
│ trialend_start_times        │ Tsd         │
│ sample_stop_times           │ Tsd         │
│ sample_start_times          │ Tsd         │
│ right_lick_times            │ Tsd         │
│ presample_stop_times        │ Tsd         │
│ presample_start_times       │ Tsd         │
│ photostim_stop_times        │ Tsd         │
│ photostim_start_times       │ Tsd         │
│ left_lick_times             │ Tsd         │
│ go_stop_times               │ Tsd         │
│ go_start_times              │ Tsd         │
│ delay_stop_times         

In [4]:
data['trials']

index    start    end           trial    photostim_onset    photostim_power    photostim_duration    trial_uid    task         task_protocol    trial_instruction    early_lick    outcome    auto_water    free_water
0        0.0      3.9999   |    1        N/A                N/A                N/A                   1            audio delay  1                left                 no early      miss       0             0
1        5.8092   9.7017   |    2        N/A                N/A                N/A                   2            audio delay  1                left                 no early      miss       0             0
2        11.4928  15.4761  |    3        N/A                N/A                N/A                   3            audio delay  1                left                 no early      miss       0             0
3        20.6713  25.4996  |    4        N/A                N/A                N/A                   4            audio delay  1                left                 no

In [17]:
data['units']

/mnt/home/svenditto/GitHub/pynapple/pynapple/core/base_class.py:42: UserWarning: Some epochs have no duration
  self.time_support = IntervalSet(start=self.index[0], end=self.index[-1])
/mnt/home/svenditto/GitHub/pynapple/pynapple/core/base_class.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  self.rate = self.index.shape[0] / np.sum(
/mnt/home/svenditto/GitHub/pynapple/pynapple/core/time_index.py:109: UserWarning: timestamps are not sorted
  warn("timestamps are not sorted", UserWarning)
/mnt/home/svenditto/GitHub/pynapple/pynapple/core/base_class.py:42: UserWarning: Some epochs have no duration
  self.time_support = IntervalSet(start=self.index[0], end=self.index[-1])
/mnt/home/svenditto/GitHub/pynapple/pynapple/core/base_class.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  self.rate = self.index.shape[0] / np.sum(
/mnt/home/svenditto/GitHub/pynapple/pynapple/core/base_class.py:42: UserWarning: Some epochs have no duration
  self.time_support

Index    rate     unit    sampling_rate    unit_quality    unit_posx    unit_posy    unit_amp            ...
-------  -------  ------  ---------------  --------------  -----------  -----------  ------------------  -----
0        2.71006  0       30000            multi           27.0         0.0          93.88322368421052   ...
1        2.03801  1       30000            good            27.0         0.0          237.17594969199172  ...
2        4.99226  2       30000            multi           59.0         0.0          103.05897887323944  ...
3        0.32445  4       30000            multi           59.0         0.0          225.53774350649348  ...
4        6.07218  5       30000            good            11.0         60.0         45.61425682507584   ...
5        0.80941  6       30000            good            43.0         20.0         119.97830840857787  ...
6        0.22446  7       30000            multi           27.0         40.0         81.5204326923077    ...
...      ...     

### Create from loaded .nwb file
Allows us to manipulate what types of objects are created, things that can't be inferred necessary from parsing the file

Grab trials as a dataframe and transform into interval set. This will be the same as what's loaded above

In [79]:
trials = nwbfile.trials.to_dataframe()
trials = trials.rename(columns={'start_time':'start','stop_time':'end'})
trials = nap.IntervalSet(trials)

{'trial': id
0        1
1        2
2        3
3        4
4        5
      ... 
529    530
530    531
531    532
532    533
533    534
Name: trial, Length: 534, dtype: int32, 'photostim_onset': id
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
529    N/A
530    N/A
531    N/A
532    N/A
533    N/A
Name: photostim_onset, Length: 534, dtype: object, 'photostim_power': id
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
529    N/A
530    N/A
531    N/A
532    N/A
533    N/A
Name: photostim_power, Length: 534, dtype: object, 'photostim_duration': id
0      N/A
1      N/A
2      N/A
3      N/A
4      N/A
      ... 
529    N/A
530    N/A
531    N/A
532    N/A
533    N/A
Name: photostim_duration, Length: 534, dtype: object, 'trial_uid': id
0        1
1        2
2        3
3        4
4        5
      ... 
529    530
530    531
531    532
532    533
533    534
Name: trial_uid, Length: 534, dtype: int32, 'task': id
0      audio delay
1      audio delay
2      a

In [80]:
trials

Index    start      end        trial    photostim_onset    photostim_power    ...
0        0.0        3.9999     1        N/A                N/A                ...
1        5.8092     9.7017     2        N/A                N/A                ...
2        11.4928    15.4761    3        N/A                N/A                ...
3        20.6713    25.4996    4        N/A                N/A                ...
4        27.8536    32.9323    5        N/A                N/A                ...
5        34.988     39.9042    6        N/A                N/A                ...
6        41.6966    46.7336    7        N/A                N/A                ...
...      ...        ...        ...      ...                ...                ...
527      3780.7522  3785.6604  528      N/A                N/A                ...
528      3788.0299  3792.9955  529      N/A                N/A                ...
529      3795.3459  3800.2825  530      N/A                N/A                ...
530      4693.22

variables saved in BehaviorEvents often have start and stop times, where the "data" value is meaningless. It would make more sense to combine them and create an interval set for each. other variables are only timestamps, with similarly meaningless "data" values associated with them. they would be better as Ts objects

grab other time stamps from BehavioralEvents and put into a dictionary. Do some manipulation to concatenate start and stop times into a dataframe

In [74]:
events = {}
beh = nwbfile.acquisition['BehavioralEvents'].time_series
for key in beh:
    if 'start' in key:
        key2 = key.replace('start_','')
        if key2 not in events.keys():
            events[key2] = pd.DataFrame(columns=['start','end'])
        events[key2]['start'] = beh[key].timestamps[:]
    elif 'stop' in key:    
        key2 = key.replace('stop_','')
        if key2 not in events.keys():
            events[key2] = pd.DataFrame(columns=['start','end'])
        events[key2]['end'] = beh[key].timestamps[:]
    else:
        events[key] = beh[key].timestamps[:]

turn into pynapple objects

In [75]:
for key in events:
    if isinstance(events[key],pd.DataFrame):
        events[key] = nap.IntervalSet(events[key])
    else:
        events[key] = nap.Ts(events[key])

### Spiking data as TsGroup

grab spiking data and import into pynapple TsGroup, preserving metadata. this should also match the pynapple loaded objects

In [86]:
units = nwbfile.units.to_dataframe()
spike_times = df["spike_times"]
metadata = df.drop(columns="spike_times")
units = nap.TsGroup(spike_times)
units.set_info(metadata)

/tmp/ipykernel_2643257/2646849493.py:4: UserWarning: Elements should not be passed as <class 'numpy.ndarray'>. Default time units is seconds when creating the Ts object.
  units = nap.TsGroup(spike_times)
/tmp/ipykernel_2643257/2646849493.py:4: UserWarning: Elements should not be passed as <class 'numpy.ndarray'>. Default time units is seconds when creating the Ts object.
  units = nap.TsGroup(spike_times)
/tmp/ipykernel_2643257/2646849493.py:4: UserWarning: Elements should not be passed as <class 'numpy.ndarray'>. Default time units is seconds when creating the Ts object.
  units = nap.TsGroup(spike_times)
/tmp/ipykernel_2643257/2646849493.py:4: UserWarning: Elements should not be passed as <class 'numpy.ndarray'>. Default time units is seconds when creating the Ts object.
  units = nap.TsGroup(spike_times)
/tmp/ipykernel_2643257/2646849493.py:4: UserWarning: Elements should not be passed as <class 'numpy.ndarray'>. Default time units is seconds when creating the Ts object.
  units = 

In [87]:
units

Index    rate     unit    sampling_rate    unit_quality    unit_posx    ...
-------  -------  ------  ---------------  --------------  -----------  -----
0        2.71006  0       30000            multi           27.0         ...
1        2.03801  1       30000            good            27.0         ...
2        4.99226  2       30000            multi           59.0         ...
3        0.32445  4       30000            multi           59.0         ...
4        6.07218  5       30000            good            11.0         ...
5        0.80941  6       30000            good            43.0         ...
6        0.22446  7       30000            multi           27.0         ...
...      ...      ...     ...              ...             ...          ...
1568     0.2113   382     30000            good            59.0         ...
1569     0.2213   383     30000            multi           11.0         ...
1570     0.00368  384     30000            multi           11.0         ...
1571     0

## Basic time series analysis
### Binning

### Tuning curves

### Bayesian decoding

# NeMoS

In [18]:
import nemos as nmo

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.


/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
